# Informer: Beyond Efficient Transformer for Long Sequence Time-Series Forecasting (AAAI'21 Best Paper)
--------------
이 실습은 Informer의 original github 코드를 기반으로 SageMaker에서 학습하는 방법을 가이드하고자 만들었습니다. 모든 라이선스는 [여기](https://github.com/zhouhaoyi/Informer2020) 구현된 원본 소스코드의 라이선스 정책을 따르고 있으며, [Informer: Beyond Efficient Transformer for Long Sequence Time-Series Forecasting](https://arxiv.org/abs/2012.07436) 논문에서 자세한 설명을 확인할 수 있습니다.

<p align="center">
<center><img src="./BlogImages/informer.png" height="90" width="450" alt=""><center>
<br><br>
<b>Figure 1.</b> The architecture of Informer.
</p>

## 1. 필요한 패키지 설치 및 업데이트

In [1]:
install_needed = True  # should only be True once
# install_needed = False

In [2]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U 'sagemaker[local]'
    !{sys.executable} -m pip install -U sagemaker-experiments smdebug
    !{sys.executable} -m pip install -U sagemaker
    !/bin/bash ./local/local_change_setting.sh
    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached sagemaker-2.91.1-py2.py3-none-any.whl
  Using cached botocore-1.24.46-py3-none-any.whl (8.7 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.24.19
    Uninstalling botocore-1.24.19:
      Successfully uninstalled botocore-1.24.19
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.86.2
    Uninstalling sagemaker-2.86.2:
      Successfully uninstalled sagemaker-2.86.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.22.97 requires botocore==1.24.42, but you have botocore 1.24.46 which is incompatible.
aiobotocore 2.0.1 requires botocore<1.22.9,>=1.22.8, but you have botocore 1.24.46 which is incompatible.
You should consider upgrading via the '/home/ec2-user/ana

## 2. 환경 설정

Sagemaker 학습에 필요한 기본적인 package를 import 합니다. <br>
[boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html)는 AWS 리소스와 동작하는 python 클래스를 제공하며, HTTP API 호출을 숨기는 추상화 모델입니다. boto3를 통해 python에서 Amazon EC2 인스턴스, S3 버켓과 같은 AWS 리소스와 동작할 수 있습니다.<br>
[sagemaker python sdk](https://sagemaker.readthedocs.io/en/stable/)는 Amazon SageMaker에서 기계 학습 모델을 교육 및 배포하기 위한 오픈 소스 라이브러리입니다.<br>

In [1]:
import matplotlib.pyplot as plt
import sagemaker
# import splitfolders

import os
import time
import warnings

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial

import boto3
import numpy as np

# from tqdm import tqdm
from time import strftime

from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch

warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'

In [2]:
role = get_execution_role()

In [3]:
sagemaker.__version__

'2.91.1'

## 3. Dataset 준비

In [4]:
# Basic data configuration is initialised and stored in the Data Preparation notebook
# ...We just retrieve it here:
%store -r
assert bucket, "Variable `bucket` missing from IPython store"
assert data_prefix, "Variable `data_prefix` missing from IPython store"

In [10]:
import pandas as pd

train_df = pd.read_csv('./data/amzforecast/target_train.csv')
test_df = pd.read_csv('./data/amzforecast/target_test.csv')
related_df = pd.read_csv('./data/amzforecast/related.csv')

In [11]:
data_df = pd.concat([train_df,test_df])

data = pd.merge(data_df, related_df, on=['timestamp', 'item_id'], how='left')
data['item_id'] = data['item_id'].astype('category')
data['item_id'] = data['item_id'].values.codes
data.rename(columns = {'timestamp':'date'},inplace=True)

In [12]:
col_list = data.columns.to_list()
new_col_list = col_list[:2] + col_list[3:] + [col_list[2]]
data = data[new_col_list]
data.head()

,date,item_id,season,holiday,weekday,workingday,weathersit,temperature,atemp,hum,windspeed,demand
0,2011-01-01 00:00:00,0,1.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.81,0.0,3.0
1,2011-01-01 00:00:00,1,1.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.81,0.0,13.0
2,2011-01-01 01:00:00,0,1.0,0.0,6.0,0.0,1.0,0.22,0.2727,0.80,0.0,8.0
3,2011-01-01 01:00:00,1,1.0,0.0,6.0,0.0,1.0,0.22,0.2727,0.80,0.0,32.0
4,2011-01-01 02:00:00,0,1.0,0.0,6.0,0.0,1.0,0.22,0.2727,0.80,0.0,5.0


In [29]:
session = boto3.Session() 
s3 = session.client(service_name="s3")

data_filename = 'informer_dataset.csv'

In [30]:
print("Writing dataframes to file...")
!mkdir -p ./data/informer

data.to_csv(
    f"./data/informer/{data_filename}",
    index=False
)

print("Uploading dataframes to S3...")
s3.upload_file(
    Filename=f"./data/informer/{data_filename}",
    Bucket=bucket,
    Key=f"{data_prefix}informer/{data_filename}"
)
dataset_path = f's3://{bucket}/{data_prefix}informer'
print(dataset_path)

print("Done")

Writing dataframes to file...
Uploading dataframes to S3...
s3://forecast-demolab-cyj-220524/data/informer
Done


## 4. Experiments 관리

Amazon SageMaker에는 실험을 관리할 수 있는 [SageMaker Experiments](https://aws.amazon.com/ko/blogs/aws/amazon-sagemaker-experiments-organize-track-and-compare-your-machine-learning-trainings/) 서비스가 있습니다. 반복적인 실험에 대해 로깅을 남기기 위한 실험 이름 (create_experiment)과 trial (create_trial) 이름을 설정하는 함수입니다. <br> 이러한 메타 정보를 이용하여 향후 ML의 실험 관리가 용이해 질 수 있습니다.

In [31]:
def create_experiment(experiment_name):
    try:
        sm_experiment = Experiment.load(experiment_name)
    except:
        sm_experiment = Experiment.create(experiment_name=experiment_name,
                                          tags=[{'Key': 'modelname', 'Value': 'informer'}])

In [32]:
def create_trial(experiment_name, i_type, i_cnt, spot=False):
    create_date = strftime("%m%d-%H%M%s")
    algo = 'informer'
    
    spot = 's' if spot else 'd'
    if i_type=='local' or i_type=='local_gpu':
        i_type = 'local'
    else:
        i_type = i_type[3:9].replace('.','-')
        
    trial = "-".join([i_type,str(i_cnt),algo, spot])
       
    sm_trial = Trial.create(trial_name=f'{experiment_name}-{trial}-{create_date}',
                            experiment_name=experiment_name)

    job_name = f'{sm_trial.trial_name}'
    return job_name

## 5. 실험 설정

학습 시 사용한 소스코드와 output 정보를 저장할 위치를 선정합니다. 이 값은 필수로 설정하지 않아도 되지만, 코드와 결과물을 S3에 저장할 때 체계적으로 정리하는데 활용할 수 있습니다.

In [33]:
code_location = f's3://{bucket}/poc_informer/sm_codes'
output_path = f's3://{bucket}/poc_informer/output' 

실험에서 표준 출력으로 보여지는 metrics 값을 정규 표현식을 이용하여 SageMaker에서 값을 capture할 수 있습니다. 이 값은 필수로 설정하지 않아도 되지만, SageMaker Experiments에 Metrics 정보를 남길 수 있어서 실험 관리에 유용합니다.

In [34]:
metric_definitions = [
    {'Name': 'Epoch', 'Regex': 'Epoch: ([-+]?[0-9]*[.]?[0-9]+([eE][-+]?[0-9]+)?),'},
    {'Name': 'train_loss', 'Regex': 'Train Loss: ([-+]?[0-9]*[.]?[0-9]+([eE][-+]?[0-9]+)?),'},
    {'Name': 'valid_loss', 'Regex': 'Valid Loss: ([-+]?[0-9]*[.]?[0-9]+([eE][-+]?[0-9]+)?),'},
    {'Name': 'test_loss', 'Regex': 'Test Loss: ([-+]?[0-9]*[.]?[0-9]+([eE][-+]?[0-9]+)?),'},
]

다양한 실험 조건을 테스트하기 위해 hyperparameters로 argument 값들을 노트북에서 설정할 수 있으며, 이 값은 학습 스크립트에서 argument인 변수로 받아서 활용이 가능합니다.

In [119]:
hyperparameters = {
        'model' : 'informer', # model of experiment, options: [informer, informerstack, informerlight(TBD)]
        'data' : 'bikeshare', # data
        'root_path' : '/', # root path of data file
        'data_path' : data_filename, # data file
        'features' : 'M', # forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate
        'target' : 'OT', # target feature in S or MS task
        'freq' : 'h', # freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h
        'checkpoints' : 'informer_checkpoints', # location of model checkpoints

        'seq_len' : 720, # input sequence length of Informer encoder
        'label_len' : 336, # start token length of Informer decoder
        'pred_len' : 336, # prediction sequence length
        # Informer decoder input: concat[start token series(label_len), zero padding series(pred_len)]

        'factor' : 5, # probsparse attn factor
        'd_model' : 512, # dimension of model
        'n_heads' : 8, # num of heads
        'e_layers' : 2, # num of encoder layers
        'd_layers' : 1, # num of decoder layers
        'd_ff' : 2048, # dimension of fcn in model
        'dropout' : 0.05, # dropout
        'attn' : 'prob', # attention used in encoder, options:[prob, full]
        'embed' : 'timeF', # time features encoding, options:[timeF, fixed, learned]
        'activation' : 'gelu', # activation
        'distil' : True, # whether to use distilling in encoder
        'output_attention' : False, # whether to output attention in ecoder
        'mix' : True,
        'padding' : 0,
        'freq' : 'h',
        'do_predict' : True,
        'batch_size' : 32,
        'learning_rate' : 0.0001,
        'loss' : 'mse',
        'lradj' : 'type1',
        'use_amp' : False, # whether to use automatic mixed precision training

        'num_workers' : 0,
        'itr' : 1,
        'train_epochs' : 1,  ## Training epochs
        'patience' : 3,
        'des' : 'exp',
#         'use_multi_gpu' : True
    }

분산학습과 spot 학습을 사용할지를 선정할 수 있습니다. <br>
분산학습의 경우 [SageMaker data parallel library](https://docs.aws.amazon.com/sagemaker/latest/dg/data-parallel.html)를 사용하고자 할 경우 distribution을 아래와 같이 설정한 후 사용할 수 있습니다. 학습 스크립트는 분산 학습 Library로 구현이 필요합니다. <br>
[spot 학습](https://docs.aws.amazon.com/sagemaker/latest/dg/model-managed-spot-training.html)을 사용하고자 할 경우 학습 파라미터에 spot 파라미터를 True로 변경한 다음, 자원이 없을 때 대기하는 시간인 max_wait (초)를 설정해야 합니다.

In [120]:
experiment_name = 'informer-poc-exp1'
distribution = None
do_spot_training = True
max_wait = None
max_run = 4*60*60

In [121]:
instance_type="ml.g5.xlarge"
# instance_type='local'
instance_count=1

## 6. 분산 설정, checkpoints 설정, 데이터 위치 설정, Local Mode 설정

### 6-1. 분산 설정

In [122]:
image_uri = None
train_job_name = 'sagemaker'


train_job_name = 'informer-dist'
distribution = {}

if instance_type in ['ml.p3.16xlarge', 'ml.p3dn.24xlarge', 'ml.p4d.24xlarge', 'local_gpu']:
    distribution["smdistributed"]={ 
                        "dataparallel": {
                            "enabled": True
                        }
                }
else:
    distribution = None

if do_spot_training:
    max_wait = max_run

print("train_job_name : {} \ntrain_instance_type : {} \ntrain_instance_count : {} \nimage_uri : {} \ndistribution : {}".format(train_job_name, instance_type, instance_count, image_uri, distribution))    

train_job_name : informer-dist 
train_instance_type : ml.g5.xlarge 
train_instance_count : 1 
image_uri : None 
distribution : None


### 6-2. checkpoints와 데이터 위치 설정, Local Mode 설정

In [155]:
from pathlib import Path

source_dir = f'{Path.cwd()}/Informer2020'
    
if instance_type =='local_gpu' or instance_type =='local':
    from sagemaker.local import LocalSession
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
    inputs = f'file://{Path.cwd()}/data/informer'

    do_spot_training = False
    checkpoint_s3_uri=None
    max_wait = None
else:
    sagemaker_session = sagemaker.Session()
    inputs = dataset_path
#     checkpoint_s3_uri = f's3://{bucket}/poc_informer/checkpoints'

## 7. 학습을 위한 Estimator 선언

AWS 서비스 활용 시 role (역할) 설정은 매우 중요합니다. 이 노트북에서 사용하는 role은 노트북과 training job을 실행할 때 사용하는 role이며, role을 이용하여 다양한 AWS 서비스에 대한 접근 권한을 설정할 수 있습니다.

In [156]:
create_experiment(experiment_name)
job_name = create_trial(experiment_name, instance_type, instance_count, do_spot_training)

In [157]:
# all input configurations, parameters, and metrics specified in estimator 
# definition are automatically tracked
estimator = PyTorch(
    entry_point='main_informer.py',
    source_dir=source_dir,
#     git_config=git_config,
    role=role,
    sagemaker_session=sagemaker_session,
    framework_version='1.10',
    py_version='py38',
    instance_count=instance_count,
    instance_type=instance_type,
#     volume_size=256,
    code_location = code_location,
    output_path=output_path,
    hyperparameters=hyperparameters,
    distribution=distribution,
    metric_definitions=metric_definitions,
    max_run=max_run,
#     checkpoint_s3_uri=checkpoint_s3_uri,
    use_spot_instances=do_spot_training,
    max_wait=max_wait,
    disable_profiler=True,
    debugger_hook_config=False,
)

## 8. 학습 수행 - 시작

In [158]:

# Now associate the estimator with the Experiment and Trial
estimator.fit(
    inputs={'training': inputs}, 
    job_name=job_name,
    experiment_config={
      'TrialName': job_name,
      'TrialComponentDisplayName': job_name,
    },
    wait=False
)

INFO:sagemaker:Creating training-job with name: informer-poc-exp1-g5-xla-1-informer-s-0523-03091653275346


In [159]:
job_name=estimator.latest_training_job.name

아래 명령어를 이용하여 시작된 학습에 대한 로그를 노트북에서 확인합니다. 이 로그는 CloudWatch에서도 확인이 가능합니다. <br> 
아래 명령어를 실행해도 학습이 시작되는 것이 아니며, 실행된 training job의 로그만 보는 것입니다.

In [160]:
sagemaker_session.logs_for_job(job_name=job_name, wait=True)

2022-05-23 03:09:08 Starting - Starting the training job...
2022-05-23 03:09:10 Starting - Launching requested ML instances......
2022-05-23 03:10:39 Starting - Preparing the instances for training......
2022-05-23 03:11:30 Downloading - Downloading input data...
2022-05-23 03:11:42 Training - Downloading the training image.....................
2022-05-23 03:15:41 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
/opt/conda/lib/python3.8/site-packages/paramiko/transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
2022-05-23 03:15:37,991 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-05-23 03:15:38,008 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-05-23 03:15:41,028 sagemaker_pytorch_container.training INFO 

## 9. 학습 결과 확인

학습이 완료된 다음 S3에 저장된 산출물을 확인합니다.<br> model 결과물은 model.tar.gz에 저장되어 있고, 이외 학습 중 로그, 결과 산출물 등은 output.tar.gz에 저장할 수 있습니다.

In [ ]:
artifacts_dir = estimator.model_data.replace('model.tar.gz', '')
print(artifacts_dir)
!aws s3 ls --human-readable {artifacts_dir}

<br> S3에 저장된 학습 결과 산출물을 모두 노트북에 다운로드 받은 다음, 압축을 풉니다.

In [ ]:
model_dir = './model'

!rm -rf $model_dir

import json , os

if not os.path.exists(model_dir):
    os.makedirs(model_dir)

!aws s3 cp {artifacts_dir}model.tar.gz {model_dir}/model.tar.gz
!tar -xvzf {model_dir}/model.tar.gz -C {model_dir}

## 10. 학습 결과의 Visualization

학습 스크립트에는 마지막 단계에 최종 학습된 모델을 이용하여 predict를 실행한 결과를 real_prediction.npy에 저장한 후 output.tar.gz로 압축하여 S3에 업로드 합니다. 이 결과를 다시 노트북에서 load한 후 plot하여 보여줍니다.

In [ ]:
import numpy as np

setting = 'informer_ETTh1_ftM_sl96_ll48_pl24_dm512_nh8_el2_dl1_df2048_atprob_fc5_ebtimeF_dtFalse_mxFalse_exp_0'

# the prediction will be saved in ./results/{setting}/real_prediction.npy
prediction = np.load(f'./model/results/{setting}/real_prediction.npy')
prediction.shape

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(prediction[0,:,-1])
plt.show()

<br>
학습 스크립트에는 마지막 단계에 최종 학습된 모델을 이용하여 테스트 데이터에 대한 추론 결과를 pred.npy에, 실제 결과는 true.npy에 저장한 후 output.tar.gz로 압축하여 S3에 업로드 합니다. 이 결과를 다시 노트북에서 load한 후 plot하여 보여줍니다.

In [ ]:
# When we finished exp.train(setting) and exp.test(setting), we will get a trained model and the results of test experiment
# The results of test experiment will be saved in ./results/{setting}/pred.npy (prediction of test dataset) and ./results/{setting}/true.npy (groundtruth of test dataset)

preds = np.load(f'./model/results/{setting}/pred.npy')
trues = np.load(f'./model/results/{setting}/true.npy')

# [samples, pred_len, dimensions]
preds.shape, trues.shape

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# draw OT (Oil Temperature) prediction
plt.figure()
plt.plot(trues[0,:,-1], label='GroundTruth')
plt.plot(preds[0,:,-1], label='Prediction')
plt.legend()
plt.show()

In [ ]:
# draw HUFL (High UseFul Load) prediction
plt.figure()
plt.plot(trues[0,:,0], label='GroundTruth')
plt.plot(preds[0,:,0], label='Prediction')
plt.legend()
plt.show()